In [ ]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-g30cwapl
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-g30cwapl
  Resolved https://github.com/ibm-granite-community/utils to commit 97732c007b2768d5c61dc61169743373043252ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":16384, "temperature":0.2},    #1024
)

In [ ]:
def zeroshot_prompt(context, question):
    """
    Creates a zero-shot prompt for the model, where the model acts as a seasoned programmer.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question or task for the model to perform
    Returns:
    - str, the formatted prompt
    """

    prompt = f"""
    You are an expert web developer. Create a complete Typing Speed Test application in a single HTML file with internal CSS and JavaScript.

    Requirements:
    1. Layout:
      - Background color: light blue (#eaf1fc)
      - Centered container with white background, rounded corners, and soft shadow
      - Top header bar:
          - On the left: website title "Typing Speed Test"
          - Right: language dropdown (options: Indonesian, English)
      - Middle section:
          - Display multiple random words (like a paragraph, not line by line)
          - Highlight the current active word with black background and white text
      - Below words: input box centered with placeholder "Type here..."
      - Below input: a green timer box (format 0:59) and a blue reset button side by side
    2. Functionality:
      - On load, display 20–30 random words from predefined word lists (Indonesian or English depending on dropdown)
      - Highlight the first word as the active word
      - Timer starts automatically on the first keystroke
      - When spacebar is pressed:
          - If input matches the active word: mark the word green and move to next
          - If input does not match: mark the word red and move to next
      - Continue until timer reaches 0
      - At the end, calculate:
          - WPM = total correct words / (time in minutes)
          - Accuracy = (total correct characters / total typed characters) * 100
      - Show results in an alert box
      - Reset button clears input, resets timer, and loads new random words
      - Prevent NaN or division by zero if no words typed
    3. Code rules:
      - All code must be in a single HTML file
      - Use only internal <style> and <script>, no external libraries or frameworks
      - Clean, responsive design similar to the given screenshot
      - Output only the HTML code, no explanations, no markdown

    Output:
    Final runnable HTML code.
    """
    return prompt


In [ ]:
def get_answer_using_zeroshot(context, question):
    """
    Generates the response from the model based on a zero-shot prompt.

    Parameters:
    - context: str, contextual information for the prompt
    - question: str, specific question for the model to answer

    Returns:
    - str, the generated result from the model
    """
    prompt = zeroshot_prompt(context, question)
    result = model.invoke(prompt)

    return result

In [ ]:
context = "Build a Typing Speed Test: display a random sentence to type, auto-start timer on first input, compute WPM and accuracy at end."
question = "Generate the full single-file HTML implementing the app."

# Generate and display the UI code for the landing page
result = get_answer_using_zeroshot(context, question)
print(f"Generated Code:\n{result}")

Generated Code:
```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Typing Speed Test</title>
<style>
  body {
    margin: 0;
    font-family: Arial, sans-serif;
    background-color: #eaf1fc;
  }
  .container {
    display: flex;
    flex-direction: column;
    align-items: center;
    padding: 20px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    background-color: white;
    border-radius: 10px;
    width: 80%;
    max-width: 600px;
  }
  header {
    display: flex;
    justify-content: space-between;
    align-items: center;
    margin-bottom: 15px;
  }
  header h1 {
    margin: 0;
  }
  .language-select {
    margin-left: 10px;
  }
  .word-container {
    display: flex;
    flex-wrap: wrap;
    justify-content: center;
    margin-bottom: 15px;
  }
  .word {
    margin: 5px;
    padding: 5px 10px;
    border-radius: 5px;
    cursor: pointer;
  }
  .active {
    background-color: black;